In [1]:
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

import glob

ImportError: No module named 'keras'

In [ ]:
weather_df = pd.read_csv('../data/weather/precip_temp.csv')
clients_glob = glob.glob('../data/wifi/**/Clients per day.csv')
clients_df = pd.concat(map(lambda csv: pd.read_csv(csv, parse_dates=[0]),
                           clients_glob), ignore_index=True)
weekday_labels = []
for d in range(7):
    label = 'weekday-%i' % d
    weekday_labels.append(label)
    clients_df[label] = [int(dt.weekday()==d) for dt in clients_df['Time']]
# Put clients date in the same format as it is in the weather data
clients_df['Time'] = [dt.strftime('%Y-%m-%d') for dt in clients_df['Time']]

all_data = clients_df.merge(weather_df, left_on='Time',
                            right_on='DATE', how='left')

prop_to_set_aside_as_test_data = 0.1
xy_test = all_data.sample(frac=prop_to_set_aside_as_test_data)
xy_train = all_data.drop(xy_test.index)

x_labels = weekday_labels + ['PRCP', 'TMAX', 'TMIN']
y_labels = ['# Clients']
x_train = xy_train[x_labels].values
y_train = xy_train[y_labels].values
x_test = xy_test[x_labels].values
y_test = xy_test[y_labels].values


In [ ]:
epochs = 40

model = Sequential()
model.add(Dense(30, activation='relu', input_shape=(10,)))
model.add(Dropout(0.2))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

model.summary()

model.compile(loss='mean_absolute_percentage_error',
              optimizer=RMSprop())

history = model.fit(x_train, y_train,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])